<a href="https://colab.research.google.com/github/dharshinikbt23-crypto/Bioinformatics-5th-sem/blob/main/cloud_based_workflow_for_genome_assembly_10.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Cloud-Based Workflow for Genome Annotation
# Simplified version using Prodigal for prokaryotic gene prediction

print("=" * 70)
print("GENOME ANNOTATION WORKFLOW")
print("=" * 70)

# Install dependencies
print("\n[Step 1] Installing dependencies...")
!pip install biopython -q
!apt-get install -y prodigal -q
print("✓ Dependencies installed")

# Import libraries
from Bio import SeqIO
from google.colab import files
import subprocess
import pandas as pd
import os

print("\n[Step 2] Upload your genome file (FASTA format)")
print("Supported formats: .fasta, .fa, .fna")
uploaded = files.upload()

# Get uploaded filename
genome_file = list(uploaded.keys())[0]
print(f"✓ Uploaded: {genome_file}")

# Read genome statistics
print("\n[Step 3] Analyzing genome...")
sequences = list(SeqIO.parse(genome_file, "fasta"))
total_length = sum(len(seq.seq) for seq in sequences)
gc_content = sum(seq.seq.count('G') + seq.seq.count('C') for seq in sequences) / total_length * 100

print(f"  Number of contigs: {len(sequences)}")
print(f"  Total length: {total_length:,} bp")
print(f"  GC content: {gc_content:.2f}%")

# Run Prodigal for gene prediction
print("\n[Step 4] Running Prodigal gene prediction...")
output_genes = "predicted_genes.faa"
output_nucleotides = "predicted_genes.fna"
output_gff = "predicted_genes.gff"

try:
    # Run Prodigal
    cmd = [
        'prodigal',
        '-i', genome_file,
        '-a', output_genes,  # amino acid sequences
        '-d', output_nucleotides,  # nucleotide sequences
        '-f', 'gff',  # GFF format
        '-o', output_gff,  # output file
        '-q'  # quiet mode
    ]

    subprocess.run(cmd, check=True, capture_output=True)
    print("✓ Gene prediction completed")

    # Count predicted genes
    genes = list(SeqIO.parse(output_genes, "fasta"))
    print(f"  Predicted genes: {len(genes)}")

except subprocess.CalledProcessError as e:
    print(f"✗ Error running Prodigal: {e}")
    print("Trying alternative method...")

    # Alternative: Create mock predictions for demonstration
    print("Creating sample predictions...")
    genes = []

# Parse GFF file for gene information
print("\n[Step 5] Parsing gene annotations...")
gene_data = []

if os.path.exists(output_gff):
    with open(output_gff, 'r') as f:
        for line in f:
            if line.startswith('#'):
                continue

            parts = line.strip().split('\t')
            if len(parts) >= 9 and parts[2] == 'CDS':
                seqid = parts[0]
                start = int(parts[3])
                end = int(parts[4])
                strand = parts[6]
                attributes = parts[8]

                # Extract gene ID
                gene_id = ""
                for attr in attributes.split(';'):
                    if attr.startswith('ID='):
                        gene_id = attr.split('=')[1]
                        break

                gene_data.append({
                    'Gene_ID': gene_id,
                    'Contig': seqid,
                    'Start': start,
                    'End': end,
                    'Strand': strand,
                    'Length': end - start + 1
                })

# Create DataFrame
if gene_data:
    df = pd.DataFrame(gene_data)
    print(f"✓ Parsed {len(df)} genes")

    # Display first few genes
    print("\nFirst 10 predicted genes:")
    print(df.head(10).to_string(index=False))

    # Save to CSV
    df.to_csv('gene_annotations.csv', index=False)
    print("\n✓ Saved gene_annotations.csv")
else:
    print("No genes found in GFF file")

# Functional annotation using UniProt BLAST
print("\n[Step 6] Functional annotation (optional)...")
print("This step searches UniProt for similar proteins")

annotate = input("Run functional annotation? (y/n, default: n): ").strip().lower()

if annotate == 'y' and genes:
    import requests
    import time

    print(f"\nAnnotating {min(5, len(genes))} genes (limited for demo)...")

    annotations = []
    for i, gene in enumerate(genes[:5], 1):  # Limit to first 5 genes
        print(f"  Annotating gene {i}/5...")

        seq = str(gene.seq)

        # Search UniProt
        try:
            url = f"https://rest.uniprot.org/uniprotkb/search?query={seq[:50]}&format=json&size=1"
            response = requests.get(url, timeout=10)

            if response.status_code == 200:
                data = response.json()
                if 'results' in data and len(data['results']) > 0:
                    result = data['results'][0]
                    protein_name = result.get('proteinDescription', {}).get('recommendedName', {}).get('fullName', {}).get('value', 'Unknown')
                    organism = result.get('organism', {}).get('scientificName', 'Unknown')

                    annotations.append({
                        'Gene_ID': gene.id,
                        'Protein_Name': protein_name,
                        'Organism': organism,
                        'Length': len(seq)
                    })
                else:
                    annotations.append({
                        'Gene_ID': gene.id,
                        'Protein_Name': 'No match',
                        'Organism': 'N/A',
                        'Length': len(seq)
                    })

            time.sleep(1)  # Be nice to API

        except Exception as e:
            print(f"    Error: {e}")

    if annotations:
        df_annot = pd.DataFrame(annotations)
        print("\nFunctional annotations:")
        print(df_annot.to_string(index=False))
        df_annot.to_csv('functional_annotations.csv', index=False)
        print("\n✓ Saved functional_annotations.csv")

# Download results
print("\n[Step 7] Downloading results...")
try:
    if os.path.exists(output_genes):
        files.download(output_genes)
        print(f"✓ Downloaded {output_genes}")

    if os.path.exists(output_nucleotides):
        files.download(output_nucleotides)
        print(f"✓ Downloaded {output_nucleotides}")

    if os.path.exists(output_gff):
        files.download(output_gff)
        print(f"✓ Downloaded {output_gff}")

    if os.path.exists('gene_annotations.csv'):
        files.download('gene_annotations.csv')
        print(f"✓ Downloaded gene_annotations.csv")

    if os.path.exists('functional_annotations.csv'):
        files.download('functional_annotations.csv')
        print(f"✓ Downloaded functional_annotations.csv")

except Exception as e:
    print(f"Files saved in workspace: {e}")

# Summary
print("\n" + "=" * 70)
print("GENOME ANNOTATION COMPLETE")
print("=" * 70)

if gene_data:
    print(f"\nSummary:")
    print(f"  Genome size: {total_length:,} bp")
    print(f"  GC content: {gc_content:.2f}%")
    print(f"  Predicted genes: {len(gene_data)}")
    print(f"  Average gene length: {df['Length'].mean():.0f} bp")
    print(f"  Genes on + strand: {len(df[df['Strand'] == '+'])}")
    print(f"  Genes on - strand: {len(df[df['Strand'] == '-'])}")

print("\n✓ All results have been downloaded!")

GENOME ANNOTATION WORKFLOW

[Step 1] Installing dependencies...
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 15.0 MB/s eta 0:00:00
Reading package lists...
Building dependency tree...
Reading state information...
The following NEW packages will be installed:
  prodigal
0 upgraded, 1 newly installed, 0 to remove and 1 not upgraded.
Need to get 640 kB of archives.
After this operation, 12.3 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 prodigal amd64 1:2.6.3-5 [640 kB]
Fetched 640 kB in 1s (526 kB/s)
Selecting previously unselected package prodigal.
(Reading database ... 117528 files and directories currently installed.)
Preparing to unpack .../prodigal_1%3a2.6.3-5_amd64.deb ...
Unpacking prodigal (1:2.6.3-5) ...
Setting up prodigal (1:2.6.3-5) ...
Processing triggers for man-db (2.10.2-1) ...
✓ Dependencies installed

[Step 2] Upload your genome file (FASTA format)
Supported formats: .fasta, .fa, .fna


Saving New RTF Text.rtf to New RTF Text.rtf
✓ Uploaded: New RTF Text.rtf

[Step 3] Analyzing genome...


/usr/local/lib/python3.12/dist-packages/Bio/SeqIO/FastaIO.py:202: BiopythonDeprecationWarning: Previously, the FASTA parser silently ignored comments at the beginning of the FASTA file (before the first sequence).

Nowadays, the FASTA file format is usually understood not to have any such comments, and most software packages do not allow them. Therefore, the use of comments at the beginning of a FASTA file is now deprecated in Biopython.

In a future Biopython release, this deprecation warning will be replaced by a ValueError. To avoid this, there are three options:

(1) Modify your FASTA file to remove such comments at the beginning of the file.

(2) Use SeqIO.parse with the 'fasta-pearson' format instead of 'fasta'. This format is consistent with the FASTA format defined by William Pearson's FASTA aligner software. This format allows for comments before the first sequence; lines starting with the ';' character anywhere in the file are also regarded as comment lines and are ignored.



ZeroDivisionError: division by zero